In [2]:
"""
Experiments with dataset in paper,
Explainable Verbal Deception Detection using Transformers
Loukas Ilias, Felix Soldner and Bennett Kleinberg
uses LIWC-15
"""
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"]  # source the ~/.zshrc file

# https://platform.openai.com/docs/guides/rate-limits/error-mitigation
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# constants, until you change them ;-)
new_line = '\n'
nb_test_samples = 20
nb_few_shot_samples_of_each_class = 8 # truth and deception
delimiter = '```\n'
# MODEL = "gpt-3.5-turbo"
# MODEL = "text-davinci-003"
# MODEL = "gpt-4"
MODEL = "gpt-4-1106-preview" # input tokes 3x cheaper and output tokens 2x cheaper than GPT-4
temperature = 1
SEED = 123

#### Get the dataset

In [3]:
import pandas as pd
df = pd.read_csv ('dataset/LIWC-15 Results - sign_events_data_statements - LIWC Analysis.csv')
# simple EDA
# print(df)
# print(df.columns)
print(f'shape: {df.shape}')  # should be 1640 x 6
# df.head


shape: (1640, 101)


#### Get the LIWC markers

In [4]:

liwc_15 = pd.read_csv ('dataset/LIWC-15 Results - sign_events_data_statements - LIWC Analysis.csv')
# simple EDA

print(f'shape: {liwc_15.shape}')  # should be 1640, 
print(liwc_15.columns)
cols = sorted(liwc_15.columns)
nb_attrib_per_line = 10
print_buf = []
for i, attrib in enumerate(cols):
    print_buf.append(attrib)
    if i % nb_attrib_per_line == 0:
        print(print_buf)
        print_buf = []

truth_markers = df[['ingest', 'bio', 'Analytic', 'number', 'leisure', 'focusfuture']]
deception_markers = df[['Apostro', 'focuspast', 'reward', 'WC', 'pronoun', 'ppron', 'Exclam', 'Tone']]
liwc_markers = df[['ingest', 'bio', 'Analytic', 'number', 'leisure', 'Apostro', 'focuspast', 'reward', 'WC', 'pronoun']]
# print(truth_markers)
# print(deception_markers)
# print(liwc_markers)

shape: (1640, 101)
Index(['signevent', 'q1', 'q2', 'unid', 'id', 'outcome_class', 'Segment', 'WC',
       'Analytic', 'Clout',
       ...
       'Colon', 'SemiC', 'QMark', 'Exclam', 'Dash', 'Quote', 'Apostro',
       'Parenth', 'OtherP', 'Emoji'],
      dtype='object', length=101)
['AllPunc']
['Analytic', 'Apostro', 'Authentic', 'Clout', 'Colon', 'Comma', 'Dash', 'Dic', 'Emoji', 'Exclam']
['OtherP', 'Parenth', 'Period', 'QMark', 'Quote', 'Segment', 'SemiC', 'Sixltr', 'Tone', 'WC']
['WPS', 'achieve', 'adj', 'adverb', 'affect', 'affiliation', 'anger', 'anx', 'article', 'assent']
['auxverb', 'bio', 'body', 'cause', 'certain', 'cogproc', 'compare', 'conj', 'death', 'differ']
['discrep', 'drives', 'family', 'feel', 'female', 'filler', 'focusfuture', 'focuspast', 'focuspresent', 'friend']
['function', 'health', 'hear', 'home', 'i', 'id', 'informal', 'ingest', 'insight', 'interrog']
['ipron', 'leisure', 'male', 'money', 'motion', 'negate', 'negemo', 'netspeak', 'nonflu', 'number']
['outcome_c

#### Some quick test to see how the truth/deceit markers are bahaving

In [13]:
import textwrap
import json
# TODO see if this can be more friendlier if GPT does not know about LIWC
def construct_liwc_attributes_json(row):
    # print('class:', 'truthful' if df.iloc[row]['outcome_class'] == 't' else 'deceptive')
    # print(liwc_markers.iloc[row])
    # print(f'q1:\n {textwrap.fill(df.iloc[row]["q1"], 100)}')
    # print(f'q2:\n {textwrap.fill(df.iloc[row]["q2"], 100)}')
    
    truthful_attributes = ['ingest', 'bio', 'Analytic', 'number', 'leisure', 'focusfuture']
    deceptive_attributes = ['Apostro', 'focuspast', 'reward', 'WC', 'pronoun', 'ppron', 'Exclam', 'Tone']
    
    freindly_truthful_attribs = ['ingestion', 'biological processes', 'analytic reasoning', 
                                 'numbers', 'leisure', 'future focus']
    freindly_deceptive_attribs = ['apostrophes', 'past focus', 'reward', 'word count', 
                                  'all pronoun', 'personal pronouns', 'axclamation marks', 'emotional tone']
                  
    # attributes = ['ingest', 'bio', 'Analytic', 'number', 'leisure', 'Apostro', 'focuspast', 'reward', 'WC', 'pronoun']
    truthful_data = {}
    for truthful_attribute, friendly_truthful_attrib in zip(truthful_attributes, freindly_truthful_attribs):
        # data[attribute] = str(row[attribute])
        truthful_data[friendly_truthful_attrib] = int(row[truthful_attribute]) 
    deceptive_data = {}
    for deceptive_attribute, friendly_deceptive_attrib in zip(deceptive_attributes, freindly_deceptive_attribs):
        # data[attribute] = str(row[attribute])
        deceptive_data[friendly_deceptive_attrib] = int(row[deceptive_attribute]) 
        
    
    return json.dumps(truthful_data), json.dumps(deceptive_data)

print(df.iloc[0])
truth_json, deception_json = construct_liwc_attributes_json(df.iloc[0].copy())
print(f'truth attributes: {truth_json}')
print(f'deception attributes: {deception_json}')
print(f"outcome: {df.iloc[0]['outcome_class']}")

signevent                                  My brothers wedding
q1           My little brother is getting married next Satu...
q2           My brother and Kate have a daughter who will b...
unid                                                  FU304384
id                                                           1
                                   ...                        
Quote                                                      0.0
Apostro                                                    0.0
Parenth                                                    0.0
OtherP                                                     0.0
Emoji                                                      0.0
Name: 0, Length: 101, dtype: object
truth attributes: {"ingestion": 0, "biological processes": 0, "analytic reasoning": 27, "numbers": 1, "leisure": 1, "future focus": 3}
deception attributes: {"apostrophes": 0, "past focus": 4, "reward": 3, "word count": 113, "all pronoun": 15, "personal pronouns": 13, "a

KeyError: 'outcome'

##### Create a random list of indices (from both classes) to be used for k-shot examples

In [60]:
def filter_by_class(df, category):
   return df[df['outcome_class']== category]

truth_df = filter_by_class(df, 't')
# print(truth_df)
print(f'truth df shape: {truth_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, truthful
truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
# print(truth_df)

deceit_df = filter_by_class(df, 'd')
# print(deceit_df)
print(f'deceit df shape: {deceit_df.shape}')  # should be 1640 x 6

# replace with a more expressive word, deceitful
deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceptive')
# print(deceit_df)

# pick random non-repeating rows
def pick_randon_non_repeating(df, quantity):
    import random
    rand_df = pd.DataFrame()
    random_list = random.sample(range(df.shape[0]), quantity)
    print("non-repeating random numbers are:")
    return df.iloc[random_list], random_list

random_truth_df, truth_indices_list = pick_randon_non_repeating(truth_df, nb_few_shot_samples_of_each_class)
# print(f'random truth list:\n {random_truth_df}')
print(f'truth indices:" {truth_indices_list}')

random_deceit_df, deceit_indices_list = pick_randon_non_repeating(deceit_df, nb_few_shot_samples_of_each_class)
# print(f'random deceit list:\n {random_deceit_df}')
deceit_indices_list = [x + truth_df.shape[0] for x in deceit_indices_list] # do this to exclude from original list
print(f'deceit indices: {deceit_indices_list}')

random_truth_deceit_df = pd.concat([random_truth_df, random_deceit_df])
few_shot_list = truth_indices_list + deceit_indices_list
print(f'truth + deceit indices(few-shot-list)\n {few_shot_list}')



truth df shape: (783, 101)
deceit df shape: (857, 101)
non-repeating random numbers are:
truth indices:" [508, 708, 698, 536, 339, 454, 756, 452]
non-repeating random numbers are:
deceit indices: [1501, 954, 1380, 1495, 1071, 806, 1440, 1038]
truth + deceit indices(few-shot-list)
 [508, 708, 698, 536, 339, 454, 756, 452, 1501, 954, 1380, 1495, 1071, 806, 1440, 1038]


/tmp/ipykernel_36845/1262882511.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truth_df['outcome_class'] = df['outcome_class'].replace('t','truthful')
/tmp/ipykernel_36845/1262882511.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deceit_df['outcome_class'] = df['outcome_class'].replace('d','deceptive')


#### Setup the OpenAI call

In [61]:
def get_chat_completion(prompt, model, temperature):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

@retry(wait=wait_random_exponential(multiplier=1, max=3), stop=stop_after_attempt(3))
def get_chat_completion_with_backoff(prompt, model, temperature):
    return get_chat_completion(prompt, model, temperature)


In [62]:
intro = f"""
You are tasked to classify the response to questions into two classes: truthful or deceptive.
You'll be presented with the following pieces of information on an activity:
(1) The title of the activity.
(2) An answer to the question: Please describe your activity. Be as specific as possible.
(3) An answer to the follow-on question: What information can you give us to reassure us that you are telling the truth?
(4) A Linguistic Inquiry Word Count (LIWC) category key and value as JSON data
Using all four pieces of information, complete the response that must be either 'truthful' or 'deceptive' and nothig else.

Here are a few examples delimited by triple backticks:

"""

response_1_heading = """Response #1:\n"""
response_2_heading = """Response #2:\n"""

liwc_header = """Linguistic Inquiry Word Count (LIWC) of responses in JSON key/value format:\n"""

def construct_activity_scenario(row):
    # activity_header = 'Title of the Activity: ' + new_line
    activity_header = 'Activity: ' + new_line
    activity_description_header = 'Question #1: Please describe your activity. Be as specific as possible.'

    activity_reassurance_header = 'Question #2: What information can you give us to reassure us that you are telling the truth?'

    activity = activity_header + row['signevent'] + 2* new_line
    q1_r1 = activity_description_header + new_line + response_1_heading + row['q1'] + 2 * new_line
    q2_r2 = activity_reassurance_header + new_line + response_2_heading + row['q2'] + 2 * new_line
    return activity + q1_r1 + q2_r2

def construct_outcome(row):
    outcome = "Are the responses truthful or deceptive?\n"
    return outcome  + row['outcome_class'] + new_line

def construct_liwc_json(row):
    pass

def construct_few_shot_prompt(few_shot_df, infer_row):
    # constructed as a list
    prompt = []
    prompt.append(intro)
    
    for _, row in few_shot_df.iterrows():
        prompt.append(delimiter)
        prompt.append(construct_activity_scenario(row))
        prompt.append(liwc_header)
        prompt.append(construct_liwc_attributes_json(row))
        prompt.append(2 * new_line)
        prompt.append(construct_outcome(row))
        prompt.append(delimiter)
        prompt.append(new_line * 2)    

    prompt.append(delimiter)
    prompt.append(construct_activity_scenario(infer_row))
    prompt.append(liwc_header)
    prompt.append(construct_liwc_attributes_json(row))
    prompt.append(2 * new_line) 
    prompt.append(construct_outcome(infer_row)) # has to have a blank outcome to be filled by the LLM
    prompt.append(delimiter)


    return prompt



In [63]:
def create_test_indices(df, total, exclude_list):
    import random
    rand_list = []
    count = 0
    while count < total:
        rand_row = random.randrange(df.shape[0])
        if rand_row not in exclude_list:
            rand_list.append(rand_row)
            count += 1
    return rand_list

In [64]:
test_indices = create_test_indices(df, nb_test_samples, few_shot_list)  # exclude the ones in the few shot list
print(f'test indices: {test_indices}')


test indices: [809, 1070, 54, 1621, 876, 1327, 677, 963, 1280, 800, 361, 1239, 103, 41, 155, 254, 36, 1244, 710, 1281]


In [65]:
y_ground_truth = []  # for computing F1-score
y_predicted = []

for i, index in enumerate(test_indices):
    infer_row = df.loc[index].copy()
    # print(f'Inferring the `class_outcome` for:\n{infer_row}')
    ground_truth = 'truthful' if infer_row['outcome_class'] == 't' else 'deceptive'
    # mask the `outcome_class` field since you want to predict it
    infer_row['outcome_class'] = ''

    # print(f'Original\n:{df.loc[index]}')
    # print(f'infer row\n: {infer_row}')

    prompt = construct_few_shot_prompt(random_truth_deceit_df, infer_row)
    prompt = ''.join(prompt)
    
    # print(f'Prompt:\n{prompt}')

    response = get_chat_completion_with_backoff(
        prompt=prompt,
        model=MODEL,
        temperature=temperature
    )    
        
    print(f'#{i}: INDEX: {index} GROUND TRUTH: {ground_truth}, RESPONSE: {response} - {"wrong" if ground_truth != response else "correct"}')
    y_ground_truth.append(ground_truth)
    y_predicted.append(response)


#0: INDEX: 809 GROUND TRUTH: deceptive, RESPONSE: deceptive - correct
#1: INDEX: 1070 GROUND TRUTH: deceptive, RESPONSE: The responses cannot be classified definitively as truthful or deceptive solely based on the information given. While the Linguistic Inquiry Word Count (LIWC) analysis may provide some insight into the cognitive and emotional states communicated in the text, it is not a foolproof method to detect deception.

The content of the responses, apparent consistency of details, and supporting information provided by individuals are factors that can contribute to the perception of truthfulness. However, these subjective interpretations cannot guarantee accuracy. Furthermore, the truthfulness of a response may be influenced by the individual's memory, communication skills, and intention.

Therefore, without additional verification methods or more context-specific indicators of truthfulness or deception, it is not possible to conclusively classify the responses based on the pro

### Compute metrics

In [66]:
from sklearn.metrics import f1_score
print(f'Total samples predicted: {len(y_predicted)}')
print(f"Weighted F1-score: {f1_score(y_ground_truth, y_predicted, average='weighted'):0.2f}")

Total samples predicted: 20
Weighted F1-score: 0.29
